### 최종

In [130]:
### 0. import
# crawling 관련
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver

# db insert 관련
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime


### 1. 법정동명 가져오기 (법정동 18,861개)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select distinct emd_cd, emd_nm from m1.pnu"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_emd = pd.DataFrame(rows)
df_emd.columns = ['emd_cd','emd_full_nm']
df_emd = df_emd[~df_emd['emd_full_nm'].isnull()] # emd_nm이 널인 지역 제외
df_emd['emd_nm'] = df_emd['emd_full_nm'].str.split(' ').str[-1] # 동이름만 추출

### 1-1. 동일한 이름의 지역 따로 처리
# 동이름 같은 지역 추출
cd_list = []
for i in df_emd.index:
    if (len(df_emd[df_emd['emd_nm']==df_emd['emd_nm'][i]]) > 1) == True:
        cd_list.append(df_emd['emd_cd'][i])
        
# (시군구 + 법정동) 명으로 변경
for i in range(len(cd_list)):
    idx = df_emd[df_emd['emd_cd']==cd_list[i]].index
    df_emd['emd_nm'][idx] = df_emd['emd_full_nm'][idx[0]].split(' ')[-2] + df_emd['emd_full_nm'][idx[0]].split(' ')[-1]
    
### 2. 법정동별 게시물 수 수집 (1차 수집)
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
# driver = webdriver.Chrome(executable_path='C:\\big14\driver\chromedriver.exe')
cnt_list = []

for dong in df_emd['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}/' # url로 태그 검색
    time.sleep(random.randrange(1,3))
    driver.get(url)
    time.sleep(random.randrange(3,5))
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        time.sleep(random.randrange(3,5))
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt'])
df1 = pd.concat([df_emd,df_cnt_list],axis=1)

# null 인 지역 (43개)
df1_null = df1[df1['cnt'].isnull()]

# 0인 지역 (16038개)
df1_0 = df1[df1['cnt'] == '0']

# null / 0인 지역 제외
df1 = df1[~df1['cnt'].isnull()]
df1 = df1[~(df1['cnt'] == '0')]

* 1차수집 : df1
* 2차수집용 : df
* 2차수집 : df2

In [18]:
### 법정동별 게시물 수 수집 (2차 수집)
# 2차 수집할 지역
df = df1[df1['cnt'].isnull()]
cnt_list = [] # 초기화

for dong in df['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}/' # url로 태그 검색
    time.sleep(random.randrange(1,3))
    driver.get(url)
    time.sleep(random.randrange(3,5))
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        time.sleep(random.randrange(3,5))
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt_1'])
df2 = pd.concat([df1,df_cnt_list],axis=1)
driver.close()

In [109]:
# cnt 한 개만 남기고 삭제
# df2 = df2.drop(columns=['cnt_1','emd_nm'])

In [119]:
# 컬럼명 변경
df2 = df2.rename(columns={'emd_full_nm':'emd_nm'})

In [230]:
df = df2

In [128]:
### 3. DB INSERT
import datetime

table = 'm1.instagram_emd_cnt'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-25 19:24:14.443821  :  0
5354


###### 기존 값이랑 매칭

In [70]:
### 수집한 값이 더 크면 값 대체
# nan to 0
idx_cnt = df2[df2['cnt']=='nan'].index
df2['cnt'][idx_cnt] = 0

idx_cnt_1 = df2[df2['cnt_1']=='nan'].index
df2['cnt_1'][idx_cnt_1] = 0

df2['cnt'] = df2['cnt'].astype(int)
df2['cnt_1'] = df2['cnt_1'].astype(int)

# 값 변경
for i in df2.index:
    if ((df2['cnt'][i] < df2['cnt_1'][i]) == True):
        df2['cnt'][i] = df2['cnt_1'][i]

In [193]:
DF2 = df2.drop(df2[df2['cnt_1'].isnull()].index)
DF2 = DF2.drop(DF2[DF2['cnt_1']=='0'].index)
DF2 = DF2.drop(columns=['cnt']).reset_index(drop=True)
DF2['cnt_1'] = DF2['cnt_1'].str.replace(',','')

In [221]:
# 수집된 값이 더 크면 업데이트 하기
for i in df2.index:
    idx = df_instargram[df2['emd_nm'][i] == df_instargram['emd_nm']].index[0]
    
    if df2['cnt_1'][i] > df_instargram['cnt'][idx]:
        df_instargram['cnt'][idx] = df2['cnt_1'][i]

In [35]:
df1 = df1.drop(df1[df1['emd_nm'].isnull()].index)
DF = df1[df1['cnt']>'0'].reset_index(drop=True)
DF['cnt'] = DF['cnt'].str.replace(',','')

In [217]:
# 수집된 값이 더 크면 업데이트 하기
for i in DF.index:
    idx = df_instargram[DF['emd_nm'][i] == df_instargram['emd_nm']].index[0]
    
    if DF['cnt'][i] > df_instargram['cnt'][idx]:
        df_instargram['cnt'][idx] = DF['cnt'][i]

###### DB에서 값 가져오기 / UPDARTE

In [186]:
### DB에서 값 받아와서 보충할 지역 재수집하기 (<0 or None인 지역만 재수집)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select * from m1.instagram_emd_cnt"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_instargram = pd.DataFrame(rows)
df_instargram.columns = ['emd_nm','emd_cd','cnt','base_ym']

In [45]:
### 받은 값 update
cnt_list = df1[df1['cnt']>'0']['cnt']
emd_cd_list = df1[df1['cnt']>'0']['emd_cd']

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

for cnt in cnt_list:
    for emd_cd in emd_cd_list:
        
        sql = f'''
        update m1.instagram_emd_cnt
           set cnt = '{cnt}'
         where emd_cd = '{emd_cd}'
        '''
        aws_cursor.execute(sql)
    
# aws_conn.commit()

KeyboardInterrupt: 

### 참고

In [9]:
### 0. import
# crawling
import time
from bs4 import BeautifulSoup
from selenium import webdriver

# db insert
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime


### 1. 법정동명 가져오기 (법정동 3968개)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select distinct emd_nm, emd_cd from m1.emd"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_emd = pd.DataFrame(rows)
df_emd.columns = ['emd_nm','emd_cd']
df_emd = df_emd.drop_duplicates('emd_nm')
df_emd['emd_cd'] = df_emd['emd_cd'] + '00'


### 2. 법정동별 게시물 수 수집 (1차 수집)
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
cnt_list = []
etc_emd_nm = [] # 추가작업 법정동

# 로그인
email = 'oasis.0416'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'localhouse'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(5)

for dong in df_emd['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}맛집/' # url로 태그 검색
    driver.get(url)
    time.sleep(3)
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        etc_emd_nm.append(dong)
        time.sleep(3)
        
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt'])
df1 = pd.concat([df_emd,df_cnt_list],axis=1)


### 법정동별 게시물 수 수집 (2차 수집)
time.sleep(10)
cnt_list = [] # 초기화
dong_list = []
etc_dong_list = []

for dong in etc_emd_nm:
    
    url = f'https://www.instagram.com/explore/tags/{dong}맛집/' # url로 태그 검색
    driver.get(url)
    time.sleep(3)
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        dong_list.append(dong)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        dong_list.append(dong)
        etc_dong_list.append(dong)
        
        time.sleep(3)
        pass

# 데이터프레임화
df2 = pd.DataFrame(data = zip(dong_list,cnt_list), columns = ['emd_nm','cnt'])
driver.close()

# 수집한 데이터 병합
df = pd.merge(df1,df2,on='emd_nm', how='left')
idx = df[df['cnt_y'].notnull()].index

for i in idx:
    df['cnt_x'][i] = df['cnt_y'][i]

# 숫자 , 제외 / base_ym 컬럼 추가
df = df.drop(columns=['cnt_y'])
df['cnt'] = df['cnt'].str.replace(',','')
df['base_ym'] = datetime.today().strftime("%Y%m")



### 3. DB INSERT
import datetime

table = 'm1.insta_emd_total_cnt'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

Exception ignored in: <function Service.__del__ at 0x000001C7A76FD820>
Traceback (most recent call last):
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\socket.py", line 

KeyboardInterrupt: 

In [ ]:
### 로그인 하면 계정 차단됨
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
driver.get('http://www.instagram.com')
driver.maximize_window()
time.sleep(3)

email = 'oasis.0416'
input_id = driver.find_elements_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')[0]
input_id.clear()
input_id.send_keys(email)

password = 'localhouse'
input_pw = driver.find_elements_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')[0]
input_pw.clear()
input_pw.send_keys(password)

input_pw.submit()
time.sleep(3)

In [46]:
# 태현님 코드

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import random

def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + str(word)
    return url
def select_first(driver):
    first = driver.find_elements_by_css_selector('div._aagw')[0]
    first.click()
    time.sleep(3)
def get_content(driver):
    try:
        contents = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]').text.split('\n')
    except:
        contents = ' '
    # 해시태그
    tags = []
    for C in contents:
        for c in C.split(' '):
            if '#' in c:
                tags.append(c)
    # 작성일
    date = contents[-1]
    data = [contents[0],date,tags]
    return data

def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
driver.get('http://www.instagram.com')
time.sleep(3)

# 로그인
email = 'th.park@ittang.co.kr'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)
password = 'kindjoker12'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(5)

# 조회할 키워드
word = input('검색어')
word = str(word)
url = insta_searching(word)

# 결과페이지 열기
driver.get(url)
time.sleep(10)

# 첫 게시물 클릭
select_first(driver)
results = []
target = 3000

for i in range(target):
    try:
        time.sleep(random.randint(40,50))
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        break
        
print(results[:2])

검색어 서귀포맛집


[[' ', ' ', []]]


In [30]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import unicodedata
import pandas as pd



# 인스타그램 키워드 검색
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/'+word
    return url

# 첫번째 요소 선택
def select_first(driver):
    time.sleep(5)
    first = driver.find_element_by_css_selector("div._aagw")
    first.click()
    time.sleep(3)

# 정보 가져오기
def get_content(driver):
    time.sleep(5)
    html = driver.page_source
    time.sleep(3)
    soup = BeautifulSoup(html,'html.parser')
    time.sleep(3)

    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC',place)
        if place == '':
            data = ['', '', '', '', '']
            return data
    except:
        place = ''

    try:
        content = soup.select('div.C4VMK')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#,\\]+', content)

    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]

    try:
        like = soup.select('div.Nm9Fw')[0].text[4:-1]
    except:
        like = 0

    data = [content, date, like, place, tags]
    return data

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

driver = webdriver.Chrome("C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe")

# 인스타그램 접속
driver.get('https://www.instagram.com/accounts/login/')
time.sleep(2)



# 로그인
email = 'oasis.0416'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'localhouse'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

# 키워드 검색
words = ['제주도맛집','제주맛집','제주도여행','제주여행']
for word in words:
    print(word)
    url = insta_searching(word)
    driver.get(url)
    time.sleep(2)

    # 검색 결과 첫 번째 선택
    select_first(driver)
    time.sleep(2)

    results = [ ]
    target = 300

    for i in range(target):
        try:
            # 내용 가져오기
            data = get_content(driver)
            if data[3] != '' :
                results.append(data)
            # 다음
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)

    results_df = pd.DataFrame(results)
    results_df.columns = ['content','data','like','place','tags']

    if word == '제주도맛집':
        results_df.to_excel('./1_crawling_jejudoMatJib.xlsx',index=False)
    elif word == '제주맛집':
        results_df.to_excel('./1_crawling_jejuMatJib.xlsx',index=False)
    elif word == '제주도여행':
        results_df.to_excel('./1_crawling_jejudoYeoHang.xlsx',index=False)
    elif word == '제주여행':
        results_df.to_excel('./1_crawling_jejuYeoHang.xlsx',index=False)

제주도맛집


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a._65Bje.coreSpriteRightPaginationArrow"}
  (Session info: chrome=106.0.5249.119)


In [ ]:
import pandas as pd

jeju_insta_df = pd.DataFrame([])

folder = './'
f_list = ['1_crawling_jejudoMatJib.xlsx',
          '1_crawling_jejuMatJib.xlsx',
          '1_crawling_jejudoYeoHang.xlsx',
          '1_crawling_jejuYeoHang.xlsx']

for frame in f_list:
    fpath = folder + frame
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns = ['content','data','like','place','tags']

jeju_insta_df.drop_duplicates(subset = ['content'], inplace = True)
jeju_insta_df.to_excel('./2_crawling_raw.xlsx', index = False)

In [ ]:
import requests
import time
import pandas as pd


def find_places(searching):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK [Kakao API Key]"
    }
    places = requests.get(url, headers = headers).json()['documents']

    place = places[0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    data = [name, x, y, searching]

    return data

raw_total = pd.read_excel('./2_crawling_raw.xlsx')
location_counts = raw_total['place'].value_counts()
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.to_excel('./3_places.xlsx')

locations = list(location_counts.index)
locations_info = [ ]

for location in locations:
    try:
        data = find_places(location)
        locations_info.append(data)
        time.sleep(0.5)
    except:
        pass
# print(locations_info)

locations_info_df = pd.DataFrame(locations_info)
locations_info_df.columns = ['카카오위치명', '경도', '위도', '인스타위치명']
locations_info_df.to_excel('./4_hotplacelocations.xlsx', index=False)

location_counts_df = pd.read_excel('./3_places.xlsx', index_col=0)
locations_info_df = pd.read_excel('./4_hotplacelocations.xlsx')

location_data = pd.merge(locations_info_df, location_counts_df,
                         how = 'inner', left_on='인스타위치명', right_index=True)

# print(location_data)

location_data = location_data.pivot_table(
    index=['카카오위치명', '경도', '위도'],
    values='place',
    aggfunc='sum'
)
location_data.to_excel('./5_hotplaceslocations_merged.xlsx')

In [ ]:
import pandas as pd
import folium

location_data = pd.read_excel('./5_hotplaceslocations_merged.xlsx')

Hanla = [33.362500, 126.533694]
map_jeju = folium.Map(location=Hanla, zoom_start=11)

for i in range(len(location_data)):
    name = location_data['카카오위치명'][i]
    count = location_data['place'][i]
    size = int(count)*2
    long = float(location_data['위도'][i])
    lat = float(location_data['경도'][i])
    folium.CircleMarker((long,lat), radius=size, color='blue', popup=name).add_to(map_jeju)
    folium.Marker(
        location=[long,lat],
        popup=name,
    ).add_to(map_jeju)

map_jeju.save('./6_hotplacesmap.html')